In [2]:
from pathlib import Path

import pandas as pd 
import numpy as np

from tqdm.notebook import tqdm

from config.config import (PROCESSED_GMT_FILE,
                           GRAPHICS_DIR, GRAPHICS_PAPER_MAIN_DIR, GRAPHICS_PAPER_APPENDIX_DIR, 
                           RESULTS_DIR,
                           fontsize_medium,
                           fontsize_small,
                           fontsize_large, 
                           regions,
                           region_naming_dict, 
                           population_shares, 
                           group_colors,
                           uncertainty_color,
                           sensitvitiy_color,
                           symbols,
                           EU27_ids,
                           WID_DATA_PROCESSED
                           )

In [6]:
magicc_df = pd.read_csv(PROCESSED_GMT_FILE, index_col = 0)

scaling_assumptions = ['CO2-scaling', 'equal-scaling', 'CH4-scaling']

In [7]:
def data_for_attributed_gmt(magicc_df, groups, group_sizes):
    # get total gmt increase since 1990 
    delta_gmt = magicc_df[magicc_df.scenario == 'ssp245'].loc[:, '2020-01-01 00:00:00'].mean()-magicc_df[magicc_df.scenario == 'ssp245'].loc[:, '1990-01-01 00:00:00'].mean()
    delta_gmt_bounds = np.quantile((magicc_df[magicc_df.scenario == 'ssp245'].loc[:, '2020-01-01 00:00:00'].values - magicc_df[magicc_df.scenario == 'ssp245'].loc[:, '1990-01-01 00:00:00'].values), q = [0.05, 0.95])
    # for storing mean increase
    group_deltas = np.zeros((len(scaling_assumptions), len(groups)))
    # for storing 5th-95th increase 
    group_delta_bounds = np.zeros((len(scaling_assumptions), len(groups), 2))
    
    for i_assumption, scaling_assumption in enumerate(scaling_assumptions):
        for i_group, group in enumerate(groups):
            group_deltas[i_assumption, i_group] = magicc_df[magicc_df.scenario == 'ssp245'].loc[:, '2020-01-01 00:00:00'].mean() - magicc_df[magicc_df.scenario == f'World_{group}_{scaling_assumption}'].loc[:, '2020-01-01 00:00:00'].mean()
            group_delta_bounds[i_assumption, i_group, :] = np.quantile((magicc_df[magicc_df.scenario == 'ssp245'].loc[:, '2020-01-01 00:00:00'].values - magicc_df[magicc_df.scenario == f'World_{group}_{scaling_assumption}'].loc[:, '2020-01-01 00:00:00'].values), q = [0.05, 0.95])
    group_deltas_fair = [delta_gmt*i for i in group_sizes]
    
    return(group_deltas, group_delta_bounds, group_deltas_fair, delta_gmt, delta_gmt_bounds)

def data_for_regional_gmt(magicc_df, regions, groups, group_sizes, population_shares):
    # full warming since 1990
    delta_gmt = magicc_df[magicc_df.scenario == 'ssp245'].loc[:, '2020-01-01 00:00:00'].mean()-magicc_df[magicc_df.scenario == 'ssp245'].loc[:, '1990-01-01 00:00:00'].mean()

    # regional data frames
    regional_group_deltas =  np.zeros((len(scaling_assumptions), len(regions), len(groups)))
    regional_group_delta_bounds = np.zeros((len(scaling_assumptions), len(regions), len(groups), 2))
    regional_group_deltas_equal = np.zeros((len(regions), len(groups)))

    
    for i_assumption, scaling_assumption in enumerate(scaling_assumptions):
        for i_region, region in enumerate(regions):
            for i_group, group in enumerate(groups): 
                regional_group_deltas[i_assumption, i_region, i_group] = magicc_df[magicc_df.scenario == 'ssp245'].loc[:, '2020-01-01 00:00:00'].mean() - magicc_df[magicc_df.scenario == f'{region}_{group}_{scaling_assumption}'].loc[:, '2020-01-01 00:00:00'].mean()
                if not ((group == 'p0p100') & (scaling_assumption != 'equal-scaling')):
                    regional_group_delta_bounds[i_assumption, i_region, i_group, :] = np.quantile(magicc_df[magicc_df.scenario == 'ssp245'].loc[:, '2020-01-01 00:00:00'].values - magicc_df[magicc_df.scenario == f'{region}_{group}_{scaling_assumption}'].loc[:, '2020-01-01 00:00:00'].values, q = [0.05, 0.95])
            
            regional_group_deltas_equal[i_region, :] = delta_gmt*population_shares[i_region]*group_sizes

    return(regional_group_deltas, regional_group_delta_bounds, regional_group_deltas_equal)

In [8]:
group_deltas, group_delta_bounds, group_deltas_fair, delta_gmt, delta_gmt_bounds = data_for_attributed_gmt(magicc_df,
                                                                                                           ['p90p100', 'p99p100', 'p999p100'],
                                                                                                           np.array([0.1,0.01,0.001]))
regional_group_deltas, regional_group_delta_bounds, regional_group_deltas_equal = data_for_regional_gmt(magicc_df, 
                                                                                                        regions,
                                                                                                        ['p0p100', 'p90p100', 'p99p100', 'p999p100'],
                                                                                                        np.array([1,0.1,0.01,0.001]),
                                                                                                        population_shares)
gmt_result_df = pd.DataFrame(columns = ['region', 'group', 
                                        'CO2-scaling_mean', 'CO2-scaling_lower-bound', 'CO2-scaling_upper-bound', 
                                        'equal-scaling_mean', 'equal-scaling_lower-bound', 'equal-scaling_upper-bound', 
                                        'equal_share',
                                        'CH4-scaling_mean', 'CH4-scaling_lower-bound', 'CH4-scaling_upper-bound', ])

row = 0
gmt_result_df.loc[row, :] = ['global', 'historic', 
                             np.nan, np.nan, np.nan, 
                             delta_gmt, delta_gmt_bounds[0], delta_gmt_bounds[1], 
                             np.nan,
                             np.nan, np.nan, np.nan,
                            ]

row += 1

for i_group, group in enumerate(['top 10', 'top 1', 'top 0.1']): 
    gmt_result_df.loc[row, :] = ['global', group, 
                                group_deltas[0, i_group], group_delta_bounds[0, i_group, 0], group_delta_bounds[0, i_group, 1],
                                group_deltas[1, i_group], group_delta_bounds[1, i_group, 0], group_delta_bounds[1, i_group, 1],
                                group_deltas_fair[i_group],
                                group_deltas[2, i_group], group_delta_bounds[2, i_group, 0], group_delta_bounds[2, i_group, 1],
                                ]
    row += 1

for i_region, region in enumerate(regions):
    for i_group, group in enumerate(['all', 'top 10', 'top 1', 'top 0.1']):
        gmt_result_df.loc[row, :] = [region, group, 
                                        regional_group_deltas[0, i_region, i_group], regional_group_delta_bounds[0, i_region, i_group, 0], regional_group_delta_bounds[0, i_region, i_group, 1], 
                                        regional_group_deltas[1, i_region, i_group], regional_group_delta_bounds[1, i_region, i_group, 0], regional_group_delta_bounds[1, i_region, i_group, 1], 
                                        regional_group_deltas_equal[i_region, i_group],
                                        regional_group_deltas[2, i_region, i_group], regional_group_delta_bounds[2, i_region, i_group, 0], regional_group_delta_bounds[2, i_region, i_group, 1]
                                        ]
        row += 1

gmt_result_df.iloc[:, 2:] = gmt_result_df.iloc[:, 2:].astype(float).round(decimals = 3)

# Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)
# gmt_result_df.to_csv(RESULTS_DIR / 'Attributed_GMT.csv')